In [1]:
import pandas as pd
from decimal import Decimal
import json
import math
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-5

In this demo we investigate the **Fast Router performance** against test orders

Initialize a fast simulator

In [2]:
filename = 'tradeBySourceAmount (2).json'
if 'Target' in filename:
    is_by_target = True
else:
    is_by_target = False
f = open(filename , "r")
bm = f.read()
bm_json = json.loads(bm)
print(len(bm_json), is_by_target)

120 False


In [3]:
def decimalize_dict(dicta):
    return({k:Decimal(v) for k,v in dicta.items()})

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['delta_liquidity'] = order0['newLiquidity'] - order0['liquidity']
    order0['y_int'] = calc_y_int(order0['liquidity'], order0['highestRate'], order0['lowestRate'], order0['marginalRate'])
    order0['fix_trade'] = order0['y_int'] - order0['liquidity']
    order0['y_int_minus_fixtrade'] = order0['y_int'] - order0['fix_trade']
    order0['order'] = order_count
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    return(order0_df)

def parse_strategy(strategy, strat_count, order_count):
    orders = pd.DataFrame()
    for order in strategy:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    # orders['pair'] = [Sim.carbon_pair.pair_slash, Sim.carbon_pair.reverse.pair_slash]
    orders['strategy'] = strat_count
    strat_count += 1
    return(orders, strat_count, order_count)


In [4]:
len(bm_json)

120

In [5]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

setup_index = 5

strats = pd.DataFrame()
strat_count = 0
order_count = 0
for strategy in bm_json[setup_index]['strategies']:
    orders, strat_count, order_count = parse_strategy(strategy, strat_count, order_count)
    strats = pd.concat([strats, orders])
strats


,liquidity,lowestRate,highestRate,marginalRate,newLiquidity,newMarginalRate,delta_liquidity,y_int,fix_trade,y_int_minus_fixtrade,order,strategy
0,869192,8.000000000000,9.000000000000,9.000000000000,870192,9.000000000000,1000,869192.000000000000000000000000000000000000000...,1E-122,869192.000000000000000000000000000000000000000...,0,0
1,2952974,1.000000000000,2.000000000000,1.996052634769,2950979,1.995264656473,-1995,2962961.99999905008983763113610455012407666987...,9987.99999905008983763113610455012407666987896...,2952974.00000000000000000000000000000000000000...,1,0
2,995648,9.000000000000,10.000000000000,10.000000000000,997648,10.000000000000,2000,995648.000000000000000000000000000000000000000...,1E-122,995648.000000000000000000000000000000000000000...,2,1
3,3926645,2.000000000000,3.000000000000,2.993323099606,3920661,2.991657471960,-5984,3950615.99999831706529839716039132851685209273...,23970.9999983170652983971603913285168520927300...,3926645.00000000000000000000000000000000000000...,3,1
4,1120104,10.000000000000,11.000000000000,11.000000000000,1123104,11.000000000000,3000,1120104.00000000000000000000000000000000000000...,0E-121,1120104.00000000000000000000000000000000000000...,4,2
5,4902306,3.000000000000,4.000000000000,3.992198220360,4890334,3.989602787675,-11972,4938270.00000090604287209463540517338877482873...,35964.0000009060428720946354051733887748287338...,4902306.00000000000000000000000000000000000000...,5,2
6,1242560,11.000000000000,12.000000000000,12.000000000000,1246560,12.000000000000,4000,1242560.00000000000000000000000000000000000000...,0E-121,1242560.00000000000000000000000000000000000000...,6,3
7,5885954,4.000000000000,5.000000000000,4.992881713007,5865990,4.989328208232,-19964,5925924.00000109098306840933411931073887565115...,39970.0000010909830684093341193107388756511579...,5885954.00000000000000000000000000000000000000...,7,3
8,1363016,12.000000000000,13.000000000000,13.000000000000,1368016,13.000000000000,5000,1363016.00000000000000000000000000000000000000...,0E-121,1363016.00000000000000000000000000000000000000...,8,4
9,6883590,5.000000000000,6.000000000000,5.995465735084,6853624,5.990936508685,-29966,6913578.00000213769752902728633905914960801369...,29988.0000021376975290272863390591496080136925...,6883590.00000000000000000000000000000000000000...,9,4


In [6]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)


Sim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='exact', raiseonerror=True)
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

for i in strats.order[::2]:
    FastSim.add_strategy('USDC', strats.y_int[i], strats.highestRate[i], strats.lowestRate[i], strats.y_int[i+1], 1/strats.highestRate[i+1], 1/strats.lowestRate[i+1])
    if strats.fix_trade[i] > 0.0000001:
        print(i)
        FastSim.amm_sells('USDC', amt=strats.fix_trade[i], execute=True, use_positions=[i], use_positions_matchlevel=[i])['trades']

for i in strats.order[1::2]:
    if strats.fix_trade[i] > 0.0000001:
        print(i)
        FastSim.amm_sells('ETH', amt=strats.fix_trade[i], execute=True, use_positions=[i], use_positions_matchlevel=[i])['trades']

initial_orders = FastSim.state()['orders']
initial_orders

1
3
5
7
9


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,874190.93559,874190.93559,USDC,False,9.00000,8.00000,9.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,2962962.00000,2952974.00000,ETH,False,0.50000,1.00000,0.50099,USDC per ETH,0
2,2,ETHUSDC,USDC,1003647.23998,1003647.23998,USDC,False,10.00000,9.00000,10.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,3950616.00000,3926645.00000,ETH,False,0.33333,0.50000,0.33408,USDC per ETH,2
4,4,ETHUSDC,USDC,1129103.78107,1129103.78107,USDC,False,11.00000,10.00000,11.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,4938270.00000,4902306.00000,ETH,False,0.25000,0.33333,0.25049,USDC per ETH,4
6,6,ETHUSDC,USDC,1250559.69644,1250559.69644,USDC,False,12.00000,11.00000,12.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,5925924.00000,5885954.00000,ETH,False,0.20000,0.25000,0.20029,USDC per ETH,6
8,8,ETHUSDC,USDC,1368015.88959,1368015.88959,USDC,False,13.00000,12.00000,13.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,6913578.00000,6883590.00000,ETH,False,0.16667,0.20000,0.16679,USDC per ETH,8


In [7]:
FastSim.state()['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,9988.00000,ETH,4998.93559,USDC,ETHUSDC,1,1,0.50049,USDC per ETH
0,0,0,A,AMM sells 9988ETH buys 4999USDC,True,True,None,9988.00000,ETH,4998.93559,USDC,ETHUSDC,[1],1,0.50049,USDC per ETH
0,1.0,1,0,route #3,False,True,None,23971.00000,ETH,7999.23998,USDC,ETHUSDC,3,1,0.33370,USDC per ETH
0,1,1,A,AMM sells 23971ETH buys 7999USDC,True,True,None,23971.00000,ETH,7999.23998,USDC,ETHUSDC,[3],1,0.33370,USDC per ETH
0,2.0,2,0,route #5,False,True,None,35964.00000,ETH,8999.78107,USDC,ETHUSDC,5,1,0.25024,USDC per ETH
0,2,2,A,AMM sells 35964ETH buys 9000USDC,True,True,None,35964.00000,ETH,8999.78107,USDC,ETHUSDC,[5],1,0.25024,USDC per ETH
0,3.0,3,0,route #7,False,True,None,39970.00000,ETH,7999.69644,USDC,ETHUSDC,7,1,0.20014,USDC per ETH
0,3,3,A,AMM sells 39970ETH buys 8000USDC,True,True,None,39970.00000,ETH,7999.69644,USDC,ETHUSDC,[7],1,0.20014,USDC per ETH
0,4.0,4,0,route #9,False,True,None,29988.00000,ETH,4999.88959,USDC,ETHUSDC,9,1,0.16673,USDC per ETH
0,4,4,A,AMM sells 29988ETH buys 5000USDC,True,True,None,29988.00000,ETH,4999.88959,USDC,ETHUSDC,[9],1,0.16673,USDC per ETH


In [8]:
bm_json[setup_index]['actions']

[{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'},
 {'strategyId': 1, 'orderIndex': 0, 'tokenAmount': '2000'},
 {'strategyId': 2, 'orderIndex': 0, 'tokenAmount': '3000'},
 {'strategyId': 3, 'orderIndex': 0, 'tokenAmount': '4000'},
 {'strategyId': 4, 'orderIndex': 0, 'tokenAmount': '5000'},
 {'strategyId': 5, 'orderIndex': 0, 'tokenAmount': '6000'}]

In [9]:
if len(bm_json[setup_index]['actions']) == 1:
    actionList = [bm_json[setup_index]['actions']]
else:
    actionList = bm_json[setup_index]['actions']
    
print(f'is_by_target {is_by_target}')
for action in actionList:
    print(action)
    targetOrder = strats.query(f"strategy=={action['strategyId']}")
    targetOrderId = targetOrder['order'].values[0]
    targetOrderToken = FastSim.state()['orders'].query(f"id=={targetOrderId}")['tkn'].values[0]
    tokenAmount = Decimal(action['tokenAmount'])
    if not is_by_target:
        if action['orderIndex'] == 1:
            FastSim.amm_buys(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        else:    
            targetOrderId += 1
            FastSim.amm_buys(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        print(targetOrderToken, targetOrderId, tokenAmount)
    else:
        if action['orderIndex'] == 1:
            FastSim.amm_sells(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        else:
            targetOrderId += 1
            FastSim.amm_sells(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        print(FastSim.carbon_pair.other(targetOrderToken), targetOrderId, tokenAmount)
    

is_by_target False
{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'}
USDC 1 1000
{'strategyId': 1, 'orderIndex': 0, 'tokenAmount': '2000'}
USDC 3 2000
{'strategyId': 2, 'orderIndex': 0, 'tokenAmount': '3000'}
USDC 5 3000
{'strategyId': 3, 'orderIndex': 0, 'tokenAmount': '4000'}
USDC 7 4000
{'strategyId': 4, 'orderIndex': 0, 'tokenAmount': '5000'}
USDC 9 5000
{'strategyId': 5, 'orderIndex': 0, 'tokenAmount': '6000'}
USDC 11 6000


In [10]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders = FastSim.state()['orders']
final_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,875190.93559,875190.93559,USDC,False,9.00000,8.00000,9.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,2962962.00000,2950978.34152,ETH,False,0.50000,1.00000,0.50119,USDC per ETH,0
2,2,ETHUSDC,USDC,1005647.23998,1005647.23998,USDC,False,10.00000,9.00000,10.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,3950616.00000,3920660.01993,ETH,False,0.33333,0.50000,0.33426,USDC per ETH,2
4,4,ETHUSDC,USDC,1132103.78107,1132103.78107,USDC,False,11.00000,10.00000,11.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,4938270.00000,4890333.29935,ETH,False,0.25000,0.33333,0.25065,USDC per ETH,4
6,6,ETHUSDC,USDC,1254559.69644,1254559.69644,USDC,False,12.00000,11.00000,12.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,5925924.00000,5865989.58157,ETH,False,0.20000,0.25000,0.20043,USDC per ETH,6
8,8,ETHUSDC,USDC,1373015.88959,1373015.88959,USDC,False,13.00000,12.00000,13.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,6913578.00000,6853623.99653,ETH,False,0.16667,0.20000,0.16692,USDC per ETH,8


In [11]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
compare_results = pd.concat([strats[['newLiquidity', 'newMarginalRate']].reset_index(drop=True), final_orders[['tkn', 'y', 'p_marg']].reset_index(drop=True)], axis=1)
compare_results.columns = ['json_liquidity', 'json_price', 'tkn', 'sim_liquidity', 'sim_price']
compare_results = pd.concat([compare_results.reset_index(drop=True), strats[['delta_liquidity']].reset_index(drop=True), (final_orders['y'] - initial_orders['y']).reset_index(drop=True)], axis=1)

compare_results = compare_results[['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_liquidity', 'y']]
compare_results.loc[:,'sim_price'] = [1/compare_results.sim_price[i] if i%2==1 else compare_results.sim_price[i] for i in compare_results.index]
# compare_results = compare_results.astype(float)
compare_results

,tkn,json_price,sim_price,json_liquidity,sim_liquidity,delta_liquidity,y
0,USDC,9.000000000000,9.00000,870192,875190.93559,1000,1000.00000
1,ETH,1.995264656473,1.99526,2950979,2950978.34152,-1995,-1995.65848
2,USDC,10.000000000000,10.00000,997648,1005647.23998,2000,2000.00000
3,ETH,2.991657471960,2.99166,3920661,3920660.01993,-5984,-5984.98007
4,USDC,11.000000000000,11.00000,1123104,1132103.78107,3000,3000.00000
5,ETH,3.989602787675,3.98960,4890334,4890333.29935,-11972,-11972.70065
6,USDC,12.000000000000,12.00000,1246560,1254559.69644,4000,4000.00000
7,ETH,4.989328208232,4.98933,5865990,5865989.58157,-19964,-19964.41843
8,USDC,13.000000000000,13.00000,1368016,1373015.88959,5000,5000.00000
9,ETH,5.990936508685,5.99094,6853624,6853623.99653,-29966,-29966.00347
